In [1]:
%load_ext autoreload
%autoreload 2

## CROP IMAGES
- Crops image pairs from stpt2imc/data/{IMC, STPT}/ and saves cropped images to disk

In [11]:
from img_processing import crop_imc, crop_stpt

for i in range(10, 11): # for loop indices correspond to the physical sections that will be processed (18)
    if i == 16:
        continue  # physical section 16 is defective
    crop_imc(i)
    crop_stpt(i)

IMC: Done physical section: 10
STPT: Done physical section: 10


## MODEL TRAINING

In [2]:
from models import UNet, PointSetGen
from datasets import STPT_IMC_ImageFolder
from fitting import train_model, validate_model

import math
import torch
import torch.nn as nn

In [3]:
use_gpu = torch.cuda.is_available()

model = UNet().double()
criterion = nn.MSELoss()

if use_gpu: 
    model = model.cuda()
    criterion = criterion.cuda()

optimizer = torch.optim.AdamW(model.parameters(), lr=0.1, weight_decay=0.01)

########### SPLIT TRAIN AND VAL ###########

img_folder = STPT_IMC_ImageFolder(root='processed_data')
train_size = math.floor(len(img_folder) * .8)
val_size = len(img_folder) - train_size
train_data, val_data = torch.utils.data.random_split(img_folder, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=64,
                                           shuffle=True)

val_loader = torch.utils.data.DataLoader(val_data,
                                         batch_size=64,
                                         shuffle=False)

In [5]:
if __name__ == '__main__':
    best_losses = 1e10
    epochs = 10

    # Train model
    for epoch in range(epochs):
        # Train for one epoch, then validate
        train_model(train_loader, model, criterion, optimizer, epoch, use_gpu)
        with torch.no_grad():
            losses = validate_model(val_loader, model, criterion, epoch, use_gpu)
        # Save checkpoint and replace old best model if current model is better
        if losses < best_losses:
            best_losses = losses
            torch.save({'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': losses,
                        'epoch': epoch,
                        'loss': losses
                       }, 'checkpoints/model-epoch-{}-losses-{:.3f}.pth'.format(epoch+1,losses))

========== Starting training epoch 0 ==========


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx